https://www.youtube.com/watch?v=mBycigbJQzA

# Part 1: Load the libraries

In [4]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [7]:
!pip install spotipy
clear_output()

In [8]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [17]:
import base64
import time
import spotipy
import requests
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
from flask import Flask, request, url_for, session, redirect
import json
from requests import post
import csv

In [10]:
## Part 1.1 Access to Spotify API

In [11]:
# Assuming client_id and client_secret are defined somewhere in your code
client_id = "d254f8fb33f04f3387783254d645fe0a"
client_secret ="3d49d97709bb40c79833aae56ec78dda"

#credmanager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
#sp = spotipy.Spotify(client_credentials_manager=credmanager)

# Part 2: Get the token 

In [12]:
# function to get the token info from the session
def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = 'https://accounts.spotify.com/api/token'
    headers = {
        'Authorization': 'Basic ' + auth_base64,
        'content-type': 'application/x-www-form-urlencoded'
      }
    data = {"grant_type": "client_credentials"}
    result = requests.post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token 

# Part 3: Working on artists info

In [13]:
def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

def search_for_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={artist_name}&type=artist&limit=1"
    query_url = url + query
    # Here we use requests.get() to make the HTTP GET request
    result = requests.get(query_url, headers=headers)
    json_result = json.loads(result.content)['artists']['items']
    if len(json_result) == 0: 
        print ("No artist with this name exists ...")
        return None
    return json_result[0]

def get_songs_by_artist(token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks?country=FR"
    headers = get_auth_header(token)
    result = requests.get(url, headers=headers)
    json_result = json.loads(result.content)["tracks"]
    return json_result

def main():
    token = get_token()
    result = search_for_artist(token, "Taylor Swift")
    if result:
        artist_id = result["id"]
        songs = get_songs_by_artist(token, artist_id)
        for idx, song in enumerate(songs):
            print (f"{idx +1}. {song['name']}")

if __name__ == "__main__":
    main()

1. Cruel Summer
2. Lover
3. Anti-Hero
4. cardigan
5. Is It Over Now? (Taylor's Version) (From The Vault)
6. Don’t Blame Me
7. august
8. Karma
9. Love Story (Taylor’s Version)
10. Blank Space


In [15]:
token = get_token()
result = search_for_artist(token, "Taylor Swift")
artist_id = result["id"]
songs = get_songs_by_artist(token, artist_id)
(result["name"])

'Taylor Swift'

In [16]:
for idx, song in enumerate(songs):
    print (f"{idx +1}. {song['name']}")

1. Cruel Summer
2. Lover
3. Anti-Hero
4. cardigan
5. Is It Over Now? (Taylor's Version) (From The Vault)
6. Don’t Blame Me
7. august
8. Karma
9. Love Story (Taylor’s Version)
10. Blank Space


# Part 4: Get the attributes of Clara Luciani

In [18]:
def read_artist_pairs_from_txt(file_path):
    artist_pairs = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            pair = line.strip().split(', ')
            artist_pairs.append(pair)
    return artist_pairs

def write_artist_info_to_csv(artist_info, file_path):
    with open(file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=artist_info[0].keys())
        writer.writeheader()
        writer.writerows(artist_info)

def main():
    # Read artist pairs from the text file
    artist_pairs = read_artist_pairs_from_txt("artist_network_clara_luciani.txt")
    
    # List to store unique artist names
    unique_artist_names = set()
    for pair in artist_pairs:
        unique_artist_names.update(pair)
    
    # List to store artist information
    all_artist_info = []
    
    # Get access token
    token = get_token()
    
    # Retrieve information for each unique artist
    for artist_name in unique_artist_names:
        # Search for the artist
        result = search_for_artist(token, artist_name)
        if result:
            artist_id = result["id"]
            
            # API URL to get artist information
            url = f"https://api.spotify.com/v1/artists/{artist_id}"
            
            # Send request to get artist information
            headers = get_auth_header(token)
            response = requests.get(url, headers=headers)
            artist_info = json.loads(response.content)
            
            # Add artist information to the list
            all_artist_info.append({
                "Artist Name": artist_info["name"],
                "Artist ID": artist_info["id"],
                "External URL": artist_info["external_urls"]["spotify"],
                "Followers": artist_info["followers"]["total"],
                "Genres": ", ".join(artist_info["genres"]),
                "Popularity": artist_info["popularity"],
                "Image URL": artist_info["images"][0]["url"] if artist_info["images"] else None
            })
    
    # Output CSV file path
    output_file_path = "Ego_attr_clara_luciani.csv"
    
    # Write artist information to a CSV file
    write_artist_info_to_csv(all_artist_info, output_file_path)
    print(f"Artist information has been written to {output_file_path}.")

if __name__ == "__main__":
    main()

Les informations des artistes ont été écrites dans artist_info.csv.


# Part 5: Get the attributes of Taylor Swift

In [26]:
def read_artist_pairs_from_txt(file_path):
    artist_pairs = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            pair = line.strip().split(', ')
            artist_pairs.append(pair)
    return artist_pairs

def write_artist_info_to_csv(artist_info, file_path):
    with open(file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=artist_info[0].keys())
        writer.writeheader()
        writer.writerows(artist_info)

def main():
    # Lecture des paires d'artistes depuis le fichier texte
    artist_pairs = read_artist_pairs_from_txt("artist_network_Taylor.txt")
    
    # Liste pour stocker les noms d'artistes uniques
    unique_artist_names = set()
    for pair in artist_pairs:
        unique_artist_names.update(pair)
    
    # Liste pour stocker les informations des artistes
    all_artist_info = []
    
    # Obtenir le jeton d'accès
    token = get_token()
    
    # Rechercher les informations de chaque artiste unique
    for artist_name in unique_artist_names:
        # Recherche de l'artiste
        result = search_for_artist(token, artist_name)
        if result:
            artist_id = result["id"]
            
            # URL de l'API pour obtenir les informations de l'artiste
            url = f"https://api.spotify.com/v1/artists/{artist_id}"
            
            # Envoi de la requête pour obtenir les informations de l'artiste
            headers = get_auth_header(token)
            response = requests.get(url, headers=headers)
            artist_info = json.loads(response.content)
            
            # Ajout des informations de l'artiste à la liste
            all_artist_info.append({
                "Artist Name": artist_info["name"],
                "Artist ID": artist_info["id"],
                "External URL": artist_info["external_urls"]["spotify"],
                "Followers": artist_info["followers"]["total"],
                "Genres": ", ".join(artist_info["genres"]),
                "Popularity": artist_info["popularity"],
                "Image URL": artist_info["images"][0]["url"] if artist_info["images"] else None
            })
    
    # Chemin du fichier CSV de sortie
    output_file_path = "Ego_attr_Taylor_Swift.csv"
    
    # Écrire les informations des artistes dans un fichier CSV
    write_artist_info_to_csv(all_artist_info, output_file_path)
    print(f"Les informations des artistes ont été écrites dans {output_file_path}.")

if __name__ == "__main__":
    main()

Les informations des artistes ont été écrites dans artist_info.csv.


In [ ]:
#2eme essai pour avoir les attr des artistes

def get_token():
    client_id = "d254f8fb33f04f3387783254d645fe0a"
    client_secret ="3d49d97709bb40c79833aae56ec78dda"
    url = 'https://accounts.spotify.com/api/token'
    headers = {
        'Authorization': 'Basic ' + base64.b64encode(f"{client_id}:{client_secret}".encode('utf-8')).decode('utf-8')
    }
    data = {"grant_type": "client_credentials"}
    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        return response.json()["access_token"]
    else:
        print("Erreur lors de la récupération du token.")
        return None

access_token = get_token()

def get_artist_id(artist_name):
    try:
        headers = {'Authorization': f'Bearer {access_token}'}
        url = f'https://api.spotify.com/v1/search?q={artist_name}&type=artist'
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            artist_data = response.json()['artists']['items']
            if artist_data:
                return artist_data[0]['id']
            else:
                print("Aucun artiste trouvé.")
                return None
        else:
            print(f"Erreur: {response.status_code}")
            return None
    except Exception as e:
        print(f"Une erreur s'est produite: {e}")
        return None

def get_artist_attributes(artist_id):
    try:
        headers = {'Authorization': f'Bearer {access_token}'}
        url = f'https://api.spotify.com/v1/artists/{artist_id}'
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Erreur: {response.status_code}")
            return None
    except Exception as e:
        print(f"Une erreur s'est produite: {e}")
        return None

def main():
    with open("artist_network_clara_luciani.txt", 'r') as file:
        artist_pairs = [line.strip().split(", ") for line in file.readlines()]

    with open("artist_attributes.csv", 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ["Artist Name", "Artist ID", "External URL", "Followers", "Genres", "Popularity", "Image URL"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for pair in artist_pairs:
            source_artist_name, _ = pair
            source_artist_id = get_artist_id(source_artist_name)
            if source_artist_id:
                artist_attributes = get_artist_attributes(source_artist_id)
                if artist_attributes:
                    writer.writerow({
                        "Artist Name": artist_attributes['name'],
                        "Artist ID": artist_attributes['id'],
                        "External URL": artist_attributes['external_urls']['spotify'],
                        "Followers": artist_attributes['followers']['total'],
                        "Genres": ", ".join(artist_attributes['genres']),
                        "Popularity": artist_attributes['popularity'],
                        "Image URL": artist_attributes['images'][0]['url'] if artist_attributes['images'] else ""
                    })
            time.sleep(5)  # Attendre un peu entre chaque requête pour éviter de dépasser les limites de l'API

if __name__ == "__main__":
    main()


# Part 6: Get attributes of the related IDs for each ego ID

In [ ]:
#1er essai : probleme dans la boucle

def get_token():
    client_id = "d254f8fb33f04f3387783254d645fe0a"
    client_secret = "3d49d97709bb40c79833aae56ec78dda"
    url = 'https://accounts.spotify.com/api/token'
    headers = {
        'Authorization': 'Basic ' + base64.b64encode(f"{client_id}:{client_secret}".encode('utf-8')).decode('utf-8')
    }
    data = {"grant_type": "client_credentials"}
    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        return response.json()["access_token"]
    else:
        print("Error retrieving the token.")
        return None

access_token = get_token()

def get_artist_related_artists(artist_id):
    try:
        headers = {'Authorization': f'Bearer {access_token}'}
        url = f'https://api.spotify.com/v1/artists/{artist_id}/related-artists'
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            artists = response.json()['artists']
            return artists
        else:
            print(f"Error: {response.status_code}")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def main():
    with open("artist_network_clara_luciani.txt", 'r') as file:
        artist_pairs = [line.strip().split(", ") for line in file.readlines()]

    with open("related_artists_attributes.csv", 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ["Ego Artist Name", "Related Artist Name", "Artist ID", "External URL", "Followers", "Genres", "Popularity", "Image URL"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for pair in artist_pairs:
            ego_artist_name, related_artist_name = pair
            # Assuming get_artist_id(ego_artist_name) is defined elsewhere
            ego_artist_id = get_artist_id(ego_artist_name)
            related_artists = get_artist_related_artists(ego_artist_id)
            if related_artists:
                for artist in related_artists:
                    writer.writerow({
                        "Ego Artist Name": ego_artist_name,
                        "Related Artist Name": artist['name'],
                        "Artist ID": artist['id'],
                        "External URL": artist['external_urls']['spotify'],
                        "Followers": artist['followers']['total'],
                        "Genres": ", ".join(artist['genres']),
                        "Popularity": artist['popularity'],
                        "Image URL": artist['images'][0]['url'] if artist['images'] else ""
                    })
            time.sleep(15)  # Wait a bit between each request to avoid exceeding API limits

if __name__ == "__main__":
    main()



In [ ]:
## Attributes of alters of Clara Luciani

In [ ]:

def get_token():
    client_id = "d254f8fb33f04f3387783254d645fe0a"
    client_secret = "3d49d97709bb40c79833aae56ec78dda"
    url = 'https://accounts.spotify.com/api/token'
    headers = {
        'Authorization': 'Basic ' + base64.b64encode(f"{client_id}:{client_secret}".encode('utf-8')).decode('utf-8')
    }
    data = {"grant_type": "client_credentials"}
    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        return response.json()["access_token"]
    else:
        print("Error while retrieving the token.")
        return None

access_token = get_token()

def get_artist_related_artists(artist_id):
    try:
        headers = {'Authorization': f'Bearer {access_token}'}
        url = f'https://api.spotify.com/v1/artists/{artist_id}/related-artists'
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            artists = response.json()['artists']
            return artists
        else:
            print(f"Error: {response.status_code}")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def main():
    with open("artist_network_clara_luciani.txt", 'r') as file:
        artist_pairs = [line.strip().split(", ") for line in file.readlines()]

    with open("related_artists_attributes_clara_luciani.csv", 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ["Ego Artist Name", "Related Artist Name", "Artist ID", "External URL", "Followers", "Genres", "Popularity", "Image URL"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for pair in artist_pairs:
            ego_artist_name, related_artist_name = pair
            ego_artist_id = get_artist_id(ego_artist_name)
            related_artists = get_artist_related_artists(ego_artist_id)
            if related_artists:
                for artist in related_artists:
                    writer.writerow({
                        "Ego Artist Name": ego_artist_name,
                        "Related Artist Name": artist['name'],
                        "Artist ID": artist['id'],
                        "External URL": artist['external_urls']['spotify'],
                        "Followers": artist['followers']['total'],
                        "Genres": ", ".join(artist['genres']),
                        "Popularity": artist['popularity'],
                        "Image URL": artist['images'][0]['url'] if artist['images'] else ""
                    })
                    time.sleep(15)  # Wait a bit between each request to avoid exceeding API limits

if __name__ == "__main__":
    main()


In [ ]:
## Attributes of alters of Taylor Swift

In [ ]:
def get_token():
    client_id = "d254f8fb33f04f3387783254d645fe0a"
    client_secret = "3d49d97709bb40c79833aae56ec78dda"
    url = 'https://accounts.spotify.com/api/token'
    headers = {
        'Authorization': 'Basic ' + base64.b64encode(f"{client_id}:{client_secret}".encode('utf-8')).decode('utf-8')
    }
    data = {"grant_type": "client_credentials"}
    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        return response.json()["access_token"]
    else:
        print("Error while retrieving the token.")
        return None

access_token = get_token()

def get_artist_related_artists(artist_id):
    try:
        headers = {'Authorization': f'Bearer {access_token}'}
        url = f'https://api.spotify.com/v1/artists/{artist_id}/related-artists'
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            artists = response.json()['artists']
            return artists
        else:
            print(f"Error: {response.status_code}")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def main():
    with open("artist_network_Taylor.txt", 'r') as file:
        artist_pairs = [line.strip().split(", ") for line in file.readlines()]

    with open("related_artists_attributes_Taylor_Swift.csv", 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ["Ego Artist Name", "Related Artist Name", "Artist ID", "External URL", "Followers", "Genres", "Popularity", "Image URL"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for pair in artist_pairs:
            ego_artist_name, related_artist_name = pair
            ego_artist_id = get_artist_id(ego_artist_name)
            related_artists = get_artist_related_artists(ego_artist_id)
            if related_artists:
                for artist in related_artists:
                    writer.writerow({
                        "Ego Artist Name": ego_artist_name,
                        "Related Artist Name": artist['name'],
                        "Artist ID": artist['id'],
                        "External URL": artist['external_urls']['spotify'],
                        "Followers": artist['followers']['total'],
                        "Genres": ", ".join(artist['genres']),
                        "Popularity": artist['popularity'],
                        "Image URL": artist['images'][0]['url'] if artist['images'] else ""
                    })
                    time.sleep(5)  # Wait a bit between each request to avoid exceeding API limits

if __name__ == "__main__":
    main()
